In [ ]:
import geotrellis.proj4.WebMercator
import geotrellis.spark._
import geotrellis.spark.io._
import geotrellis.spark.io.file._
import geotrellis.spark.io.hadoop._
import geotrellis.spark.io.index.ZCurveKeyIndexMethod
import geotrellis.spark.ingest.MultibandIngest
import geotrellis.spark.tiling.ZoomedLayoutScheme
import geotrellis.vector.ProjectedExtent

In [ ]:
val sourcePath = "file:///home/tbeatty/ramas-data/nbcommercial/40/2018-06-22-rgb.tif"
val sourceRdd = sc.hadoopMultibandGeoTiffRDD(sourcePath)

In [ ]:
val catalogPath = "/tmp/geotrellis-catalog"

val layerWriter = FileLayerWriter(catalogPath)
val layerDeleter = FileLayerDeleter(catalogPath)

In [ ]:
val layerName = "example"
val maxZoom = 16

MultibandIngest[ProjectedExtent, SpatialKey](
  sourceRdd,
  WebMercator,
  ZoomedLayoutScheme(WebMercator, 256),
  pyramid = true,
  maxZoom = Some(maxZoom)
) { (rdd, zoom) =>
  val layerId = LayerId(layerName, zoom)
  if (layerWriter.attributeStore.layerExists(layerId)) {
    println(s"Removing existing layer $layerId")
    layerDeleter.delete(layerId)
  }
  println(s"Writing $layerId")
  layerWriter.write(layerId, rdd, ZCurveKeyIndexMethod)
}